In [0]:
spark.conf.set('spark.databricks.io.cache.enabled', False)

In [0]:
spark.version
##spark.conf.get("spark.databricks.clusterUsageTags.clusterName", "Not Databricks")
##spark.conf.get("spark.databricks.io.cache.enabled", "Not Databricks")
spark.conf.get("spark.databricks.version")

In [0]:
from pyspark.sql.functions import *

df = (
    spark
        .range(0,150000000000,1,32)
        .select(
            hash('id').alias('id'),
            rand().alias('value'),
            from_unixtime(lit(1701692381+col('id'))).alias('time')
        )
)

##df.display()

##df2 = (spark.range(0,2500))
##df2.display()

In [0]:
from pyspark.sql.functions import *

df = (
    spark
        .range(0,150000000000,1)#,32)
        .select(
            'id',
            round(rand()*1000,2).alias('amount'),
            (col('id')%10).alias('country_id'),
            (col('id')%100).alias('store_id'),
            round(rand()*100000000,0).alias('customer_id'),
            from_unixtime(lit(1701692381+col('id'))).alias('time')
        )
)

#df.display()

##df2 = (spark.range(0,2500))
##df2.display()

In [0]:
(df
 .write
 .mode('overwrite')
 .option('overwriteSchema',"true")
 #.option('mergeSchema',"true")
 #.partitionBy('id')
 .saveAsTable('transactions')
 )

In [0]:
store_df= (spark
            .range(0,99)
            .select(
                'id',
                round(rand()*100,0).alias('employees'),
                (col('id')%10).alias('country_id'),
                expr('uuid()').alias('name')
            ))
store_df.display()

store_df.write.saveAsTable('stores')


In [0]:
from pyspark.sql.functions import col, round, rand, expr, when

store_df = (
    spark
        .range(0, 100000000)
        .select(
            'id',
            round(col('id')%100000,0).alias('customer_master_id'),
            round(rand() * 100, 0).alias('customer_band'),
            (col('id') % 10).alias('country_id'),
            expr('uuid()').alias('name'),
            when(round(col('id')%100000,0) == col('id'), True)
                .otherwise(False)
                .alias('IsCurrent')
        )
)

store_df.display()

store_df.write.mode("overwrite").option('overwriteSchema','true').saveAsTable("customers")


In [0]:
%sql
--select * from customers c
--inner join customers cc on c.customer_master_id=cc.id --and cc.IsCurrent=true
--where  c.id=2000050

select * from customers where customer_master_id=50


In [0]:
countries = [
    (0,"Italy"),
    (1,"France"),
    (2,"Spain"),
    (3,"Germany"),
    (4,"UK"),
    (5,"USA"),
    (6,"Canada"),
    (7,"Mexico"),
    (8,"Brazil"),
    (9,"Argentina"),
    (10,"China"),
    (11,"Japan"),
    (12,"Korea"),
    (13,"India"),
    (14,"Australia"),
    (15,"New Zealand")
]

columns = ["id","name"]

countries_df = spark.createDataFrame(data=countries,schema=columns  )

countries_df.display()

countries_df.write.saveAsTable('countries')

In [0]:
%sql

select * from iot_data where id=1158307975

In [0]:
%sql

select  avg(value) from iot_data where time>="2023-12-04 12:19:00" and time<="2023-12-04 13:01:20"

In [0]:
%sql

--select _metadata.file_path as filename, min(id),max(id) from iot_data group by _metadata.file_path;

select _metadata.file_path as filename, min(time),max(time) from iot_data group by _metadata.file_path

In [0]:
%sql

SET use_cached_result = false

In [0]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold",-1)
spark.conf.set("spark.databricks.adaptive.autoBroadcastJoinThreshold",-1)


In [0]:
joined_df = spark.sql('''
select transactions.id,
    amount,
    countries.name as country_name,
    employees,
    stores.name as store_name,
    max(cc.name) max_customer_name,
    max(cc.customer_band) max_customer_band
from transactions as transactions
left join /*+ BROADCAST(stores) */
    stores on
        transactions.store_id=stores.id
left join /*+ BROADCAST(stores) */
    countries on 
        transactions.country_id=countries.id
left join /*+ BROADCAST(stores) */
    customers on
        transactions.customer_id=customers.id
left join 
    customers cc on
        customers.customer_master_id=cc.id
group by transactions.id,
    amount,
    countries.name,
    employees,
    stores.name 
''')

joined_df.write.mode('overwrite').option('mergeSchema','true').saveAsTable('transactioned_countries')

In [0]:
joined_df = spark.sql('''
select transactions.id,
    amount,
    countries.name as country_name,
    employees,
    stores.name as store_name,
    max(cc.name) max_customer_name,
    max(cc.customer_band) max_customer_band
from transactions as transactions
left join /*+ BROADCAST(stores) */
    stores on
        transactions.store_id=stores.id
left join /*+ BROADCAST(stores) */
    countries on 
        transactions.country_id=countries.id
left join /*+ BROADCAST(stores) */
    customers on
        transactions.customer_id=customers.id
left join 
    customers cc on
        customers.customer_master_id=cc.id
where stores.employees > 10 and stores.employees<=35 and customers.band between 15 and 65
group by transactions.id,
    amount,
    countries.name,
    employees,
    stores.name 
''')

joined_df.write.mode('overwrite').option('mergeSchema','true').saveAsTable('transactioned_countries22')

In [0]:
store_df= (spark
            .range(0,99)
            .select(
                'id',
                round(rand()*100,0).alias('employees'),
                (col('id')%10).alias('country_id'),
                expr('uuid()').alias('name')
            ))
store_df.display()

store_df.write.saveAsTable('stores')


In [0]:
%sql

select * from countries

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, LongType, ArrayType, TimestampType, BigIntType,UnixTimestampType,TimestampType
from pyspark.sql.functions import current_timestamp

schema = StructType([StructField('id',BigIntType(),False),
                    StructField('amount',DoubleType(),False),
                    StructField('country_id',BigIntType(),False),
                    StructField('store_id',BigIntType(),False),
                    StructField('customer_id',BigIntType(),False),
                    StructField('time',UnixTimestampType,False),
                    StructField('timestamp',TimestampType(),False)
                    ])
empty_df = spark.createDataFrame([],schema=schema)
display(empty_df)


#empty_df= empty_df.withColumn('timestamp',current_timestamp())

empty_df.write.mode("merge").option("mergeSchema","true").saveAsTable(name="transactions")



In [0]:
%sql
--2h
--select _metadata.file_path as filename, min(id),max(id) from iot_data group by _metadata.file_path;

select _metadata.file_path as filename,_metadata.file_modification_time as file_modification_time,_metadata.file_name as file_name, min(id) as min_id,max(id) as max_id,min(country_id) as min_country_id,max(country_id) as max_country_id,min(store_id) as min_store_id,max(store_id) as max_store_id,
min(customer_id) as min_customer_id,max(customer_id) as max_customer_id,count(*) as count_rc
from transactions group by _metadata.file_path,_metadata.file_modification_time,_metadata.file_name

In [0]:
spark.conf.set('spark.databricks.execution.timeout', timeout_in_seconds).